# Testing gradients.py
Author: Thomas Patton

The goal of this notebook is to demonstrate how gradients.py can be used to give explanation for a question-answering model

## Imports / Installs

In [1]:
!pip install -q transformers ipywidgets jupyter tensorflow

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from IPython.core.display import display, HTML
from transformers import AutoTokenizer, TFBertForQuestionAnswering

## Load BERT Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("./bert_model/")
model = TFBertForQuestionAnswering.from_pretrained("./bert_model/", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


## Define Question / Context

In [4]:
with open('./sample.txt') as f:
    context = f.read()[500:1500]
    
context = context[500:750] + ' the patient needs l3 l4 laminectomies. ' + context[750:1500]
        
question = "Does the paitent need a laminectomy?"
print(context)

one physical therapy and this made him worse. he has been offered injections but is refused these. he is 
concerned about his diabetes. i have offered him injection several times. he does not wish to have any. 
physical examination: 
alert and orient the patient needs l3 l4 laminectomies. ed x 4. patient in no acute distress. normal affect and mentation. 
the patient stands with the shoulders and pelvis level. sagittal and frontal contours of the spine are well maintained. 
examination of the low back region reveals the skin to be int


## Get Gradients

In [5]:
%load_ext autoreload

In [6]:
%autoreload 2

In [7]:
import gradients as grad

gradient_df = grad.get_gradients(model, tokenizer, question, context, verbose=True)
gradient_df

the patient needs l3 l4 laminectomies. ed x 4. patient in no acute distress


,word,gradient,type
0,[CLS],0.123764,0
1,Does,0.421226,0
2,the,0.146855,0
3,p,0.322579,0
4,##ait,1.000000,0
...,...,...,...
129,to,0.022984,1
130,be,0.024991,1
131,in,0.028921,1
132,##t,0.031556,1


## Visualize Gradients

In [8]:
from IPython.core.display import display, HTML

text = grad.html(gradient_df['word'].tolist(), gradient_df['gradient'].tolist(), thresh=0.25)
display(HTML(text))